In [22]:
import pandas as pd
import numpy as np
import json
import gc
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import pytorch_lightning as pl
import albumentations
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from albumentations.core.composition import Compose
from albumentations.pytorch import ToTensorV2
import torchvision
from typing import Any, Dict, List, Union, Optional
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import StratifiedKFold
import cv2
import torch
from efficientnet_pytorch import EfficientNet
import numpy as np

from PIL import Image
from PIL import ImageFile

In [23]:
import pytorch_lightning
pytorch_lightning.__version__

'1.1.0'

In [24]:
base_dir = r'C:\Users\Kaggle\Leaf_Classification\input'

In [25]:
### read csv and label mapping

labels = pd.read_csv(os.path.join(base_dir,'train.csv'))

with open(f'{base_dir}/label_num_to_disease_map.json') as f:
    label_mapper = json.load(f)

In [26]:
weights = (labels['label'].value_counts(normalize=True,ascending=False)).to_dict()

In [27]:
weights

{3: 0.6149460204701593,
 4: 0.12043744450156564,
 2: 0.11151095948030097,
 1: 0.10230406131700706,
 0: 0.05080151423096696}

#### Dataset definition

In [28]:
class ImageClassificationDataset(Dataset):
    def __init__(
        self,
        image_names: List,
        transforms: Compose,
        labels: Optional[List[int]],
        img_path: str = '',
        mode: str = 'train',
        labels_to_ohe: bool = False,
        n_classes: int = 5,
    ):
        """
        Image classification dataset.

        Args:
            df: dataframe with image id and bboxes
            mode: train/val/test
            img_path: path to images
            transforms: albumentations
        """

        self.mode = mode
        self.transforms = transforms
        self.img_path = img_path
        self.image_names = image_names
        if labels is not None:
            if not labels_to_ohe:
                self.labels = np.array(labels)
            else:
                self.labels = np.zeros((len(labels), n_classes))
                self.labels[np.arange(len(labels)), np.array(labels)] = 1

    def __getitem__(self, idx: int) -> Dict[str, np.array]:
        image_path = self.img_path + self.image_names[idx]
        image = cv2.imread(f'{image_path}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if image is None:
            raise FileNotFoundError(image_path)
        target = self.labels[idx]

        img = self.transforms(image=image)['image']
        sample = {'image_path': image_path, 'image': img, 'target': np.array(target).astype('int64')}

        return sample

    def __len__(self) -> int:
        return len(self.image_names)


### Define augmentations

In [29]:
sz = 576

In [30]:
# augmentations taken from: https://www.kaggle.com/khyeh0719/pytorch-efficientnet-baseline-train-amp-aug
train_augs = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.CoarseDropout(p=0.5),
            albumentations.Cutout(p=0.5), 
            ToTensorV2()],
            p=1.)
  
        
valid_augs = albumentations.Compose([
            albumentations.CenterCrop(sz, sz, p=1.),
            albumentations.Resize(sz, sz),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0)
            ,ToTensorV2()], p=1.)

In [31]:
# # augmentations taken from: https://www.kaggle.com/khyeh0719/pytorch-efficientnet-baseline-train-amp-aug
# train_augs = albumentations.Compose([
#             albumentations.RandomResizedCrop(256, 256),
#             albumentations.Transpose(p=0.5),
#             albumentations.HorizontalFlip(p=0.5),
#             albumentations.VerticalFlip(p=0.5),
#             albumentations.ShiftScaleRotate(p=0.5),
#             albumentations.HueSaturationValue(
#                 hue_shift_limit=0.2, 
#                 sat_shift_limit=0.2, 
#                 val_shift_limit=0.2, 
#                 p=0.5
#             ),
#             albumentations.RandomBrightnessContrast(
#                 brightness_limit=(-0.1,0.1), 
#                 contrast_limit=(-0.1, 0.1), 
#                 p=0.5
#             ),
#             albumentations.Normalize(
#                 mean=[0.485, 0.456, 0.406], 
#                 std=[0.229, 0.224, 0.225], 
#                 max_pixel_value=255.0, 
#                 p=1.0
#             ),
#             albumentations.CoarseDropout(p=0.5),
#             albumentations.Cutout(p=0.5), 
#             ToTensorV2()],
#             p=1.)
  
        
# valid_augs = albumentations.Compose([
#             albumentations.CenterCrop(256, 256, p=1.),
#             albumentations.Resize(256, 256),
#             albumentations.Normalize(
#                 mean=[0.485, 0.456, 0.406], 
#                 std=[0.229, 0.224, 0.225], 
#                 max_pixel_value=255.0, 
#                 p=1.0)
#             ,ToTensorV2()], p=1.)

### PL Data Module - https://www.kaggle.com/artgor/cassava-disease-identification-with-lightning

In [32]:
class CassavaDataModule(pl.LightningDataModule):
    def __init__(self,
                 df,
                 train_augs,
                 valid_augs,
                 path,
                bs=8,
                fold=0):
        super().__init__()
        self.df = df
        self.train_augs = train_augs
        self.valid_augs = valid_augs
        self.path = path
        self.bs = bs
        self.fold = fold

    def prepare_data(self):
        pass

    def setup(self, stage=None):
        
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        
        train_indexes, valid_indexes = list(folds.split(self.df, self.df['label']))[self.fold]
        
        train_df = self.df.iloc[train_indexes]
        valid_df = self.df.iloc[valid_indexes]

        
        self.train_dataset = ImageClassificationDataset(image_names=train_df['image_id'].values,
                                                        transforms=train_augs,
                                                        labels=train_df['label'].values,
                                                        img_path=self.path,
                                                        mode='train',
                                                        labels_to_ohe=False,
                                                        n_classes=5)
        self.valid_dataset = ImageClassificationDataset(image_names=valid_df['image_id'].values,
                                                        transforms=valid_augs,
                                                        labels=valid_df['label'].values,
                                                        img_path=self.path,
                                                        mode='valid',
                                                        labels_to_ohe=False,
                                                        n_classes=5)

    def train_dataloader(self):
        train_loader = torch.utils.data.DataLoader(
            self.train_dataset,
            batch_size=self.bs,
            num_workers=0,
            shuffle=True,
        )
        return train_loader

    def val_dataloader(self):
        valid_loader = torch.utils.data.DataLoader(
            self.valid_dataset,
            batch_size=self.bs,
            num_workers=0,
            shuffle=False,
        )

        return valid_loader

    def test_dataloader(self):
        return None

### Define encoder - https://www.kaggle.com/artgor/cassava-disease-identification-with-lightning

In [33]:
def freeze_until(net: Any, param_name: str = None) -> None:
    """
    Freeze net until param_name


    Args:
        net:
        param_name:

    """
    found_name = False
    for name, params in net.named_parameters():
        if name == param_name:
            found_name = True
        params.requires_grad = found_name

class BasicEncoder(nn.Module):
    def __init__(
        self,
        arch: str = 'resnet18',
        source: str = 'torchvision',
        pretrained: str = None,
        n_layers: int = -2,
        freeze: bool = False,
        to_one_channel: bool = False,
        freeze_until_layer: str = None,
    ) -> None:
        """
        Initialize Encoder.

        Args:
            num_classes: the number of target classes, the size of the last layer's output
            arch: the name of the architecture form pretrainedmodels
            pretrained: the mode for pretrained model from pretrainedmodels
            n_layers: number of layers to keep
            freeze: to freeze model
            freeze_until: freeze until this layer. If None, then freeze all layers
        """
        super().__init__()
        if 'eff' in arch:
            net = EfficientNet.from_pretrained(arch)
            self.output_dimension = net._fc.in_features
        elif source == 'pretrainedmodels':
            net = pretrainedmodels.__dict__[arch](pretrained=pretrained)
            self.output_dimension = list(net.children())[-1].in_features
        elif source == 'torchvision':
            net = torchvision.models.__dict__[arch](pretrained=pretrained)
            #net.load_state_dict(torch.load(f'../input/pytorch-pretrained-image-models/{arch}.pth'))
            self.output_dimension = list(net.children())[-1].in_features
            
        if freeze:
            freeze_until(net, freeze_until_layer)
        
        if 'eff' in arch:
            n_layers = min(n_layers,-4)
            layers = list(net.children())[:n_layers]
            print("Effnet loop")
        elif ((n_layers is not None) and ('eff' not in arch)) :
            layers = list(net.children())[:n_layers]
            print("Not effnet loop")
        else:
            layers = net
            print("all else")
            
        if to_one_channel:
            # https://www.kaggle.com/c/bengaliai-cv19/discussion/130311#745589
            # saving the weights of the first conv in w
            w = layers[0].weight
            # creating new Conv2d to accept 1 channel
            layers[0] = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
            # substituting weights of newly created Conv2d with w from but we have to take mean
            # to go from  3 channel to 1
            layers[0].weight = nn.Parameter(torch.mean(w, dim=1, keepdim=True))
            layers = nn.Sequential(*layers)

        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        output = self.layers(x)

        return output

In [34]:
class BasicDecoder(nn.Module):
    def __init__(self, pool_output_size: int = 1, n_classes: int = 1, output_dimension: int = 512) -> None:
        """
        Initialize Decoder.

        Args:
            pool_output_size: the size of the result feature map after adaptive pooling layer
            n_classes: n classes to output
            output_dimension: output dimension of encoder
        """
        super().__init__()
        self.pool = nn.AdaptiveAvgPool2d(output_size=pool_output_size)
        #self.fc = nn.Linear(output_dimension * pool_output_size * pool_output_size, n_classes)
        self.fc = nn.Linear(output_dimension , n_classes)

    def forward(self, x):
        x = self.pool(x)
        output = self.fc(x.view(x.size()[0], -1))

        return output

In [35]:
#list(EfficientNet.from_pretrained('efficientnet-b0').children())[:]

In [36]:
# _ = EfficientNet.from_pretrained('efficientnet-b0')
# _._fc.in_features

In [37]:
#list(torchvision.models.__dict__['resnext101_32x8d'](pretrained=True).children())[:-2]

In [38]:
class CosineLoss(nn.Module):
    def __init__(self, xent=.1, reduction="mean"):
        super(CosineLoss, self).__init__()
        self.xent = xent
        self.reduction = reduction
        
        self.y = torch.Tensor([1]).cuda()
        
    def forward(self, input, target):
        cosine_loss = F.cosine_embedding_loss(input, F.one_hot(target, num_classes=input.size(-1)), self.y, reduction=self.reduction)
        cent_loss = F.cross_entropy(F.normalize(input), target, reduction=self.reduction)
        
        return cosine_loss + self.xent * cent_loss

class FocalCosineLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, xent=.1, reduction="mean"):
        super(FocalCosineLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

        self.xent = xent
        self.reduction = reduction
        
        self.y = torch.Tensor([1]).cuda()
        
    def forward(self, input, target):
        cosine_loss = F.cosine_embedding_loss(input, F.one_hot(target, num_classes=input.size(-1)), self.y, reduction=self.reduction)

        cent_loss = F.cross_entropy(F.normalize(input), target, reduce=False)
        pt = torch.exp(-cent_loss)
        focal_loss = self.alpha * (1-pt)**self.gamma * cent_loss

        if self.reduction == "mean":
            focal_loss = torch.mean(focal_loss)
        
        return cosine_loss + self.xent * focal_loss

In [39]:
### https://github.com/AdeelH/pytorch-multi-class-focal-loss/blob/master/focal_loss.py

from typing import Optional, Sequence

import torch
from torch import Tensor
from torch import nn
from torch.nn import functional as F

class FocalLoss(nn.Module):
    """ Focal Loss, as described in https://arxiv.org/abs/1708.02002.
    It is essentially an enhancement to cross entropy loss and is
    useful for classification tasks when there is a large class imbalance.
    x is expected to contain raw, unnormalized scores for each class.
    y is expected to contain class labels.
    Shape:
        - x: (batch_size, C) or (batch_size, C, d1, d2, ..., dK), K > 0.
        - y: (batch_size,) or (batch_size, d1, d2, ..., dK), K > 0.
    """

    def __init__(self,
                 alpha: Optional[Tensor] = None,
                 gamma: float = 0.,
                 reduction: str = 'mean',
                 ignore_index: int = -100):
        """Constructor.
        Args:
            alpha (Tensor, optional): Weights for each class. Defaults to None.
            gamma (float, optional): A constant, as described in the paper.
                Defaults to 0.
            reduction (str, optional): 'mean', 'sum' or 'none'.
                Defaults to 'mean'.
            ignore_index (int, optional): class label to ignore.
                Defaults to -100.
        """
        if reduction not in ('mean', 'sum', 'none'):
            raise ValueError(
                'Reduction must be one of: "mean", "sum", "none".')

        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ignore_index = ignore_index
        self.reduction = reduction

        self.nll_loss = nn.NLLLoss(
            weight=torch.tensor(alpha), reduction='none', ignore_index=ignore_index)

    def __repr__(self):
        arg_keys = ['alpha', 'gamma', 'ignore_index', 'reduction']
        arg_vals = [self.__dict__[k] for k in arg_keys]
        arg_strs = [f'{k}={v}' for k, v in zip(arg_keys, arg_vals)]
        arg_str = ', '.join(arg_strs)
        return f'{type(self).__name__}({arg_str})'

    def forward(self, x: Tensor, y: Tensor) -> Tensor:
        if x.ndim > 2:
            # (N, C, d1, d2, ..., dK) --> (N * d1 * ... * dK, C)
            c = x.shape[1]
            x = x.permute(0, *range(2, x.ndim), 1).reshape(-1, c)
            # (N, d1, d2, ..., dK) --> (N * d1 * ... * dK,)
            y = y.view(-1)

        unignored_mask = y != self.ignore_index
        y = y[unignored_mask]
        if len(y) == 0:
            return 0.
        x = x[unignored_mask]

        # compute weighted cross entropy term: -alpha * log(pt)
        # (alpha is already part of self.nll_loss)
        log_p = F.log_softmax(x, dim=-1)
        ce = self.nll_loss(log_p, y)

        # get true class column from each row
        all_rows = torch.arange(len(x))
        log_pt = log_p[all_rows, y]

        # compute focal term: (1 - pt)^gamma
        pt = log_pt.exp()
        focal_term = (1 - pt)**self.gamma

        # the full loss: -alpha * ((1 - pt)^gamma) * log(pt)
        loss = focal_term * ce

        if self.reduction == 'mean':
            loss = loss.mean()
        elif self.reduction == 'sum':
            loss = loss.sum()

        return loss


def focal_loss(alpha: Optional[Sequence] = None,
               gamma: float = 0.,
               reduction: str = 'mean',
               ignore_index: int = -100,
               device='cpu',
               dtype=torch.float32) -> FocalLoss:
    """Factory function for FocalLoss.
    Args:
        alpha (Sequence, optional): Weights for each class. Will be converted
            to a Tensor if not None. Defaults to None.
        gamma (float, optional): A constant, as described in the paper.
            Defaults to 0.
        reduction (str, optional): 'mean', 'sum' or 'none'.
            Defaults to 'mean'.
        ignore_index (int, optional): class label to ignore.
            Defaults to -100.
        device (str, optional): Device to move alpha to. Defaults to 'cpu'.
        dtype (torch.dtype, optional): dtype to cast alpha to.
            Defaults to torch.float32.
    Returns:
        A FocalLoss object
    """
    if alpha is not None:
        if not isinstance(alpha, Tensor):
            alpha = torch.tensor(alpha)
        alpha = alpha.to(device=device, dtype=dtype)

    fl = FocalLoss(
        alpha=alpha,
        gamma=gamma,
        reduction=reduction,
        ignore_index=ignore_index)
    return fl

In [40]:
weights

{3: 0.6149460204701593,
 4: 0.12043744450156564,
 2: 0.11151095948030097,
 1: 0.10230406131700706,
 0: 0.05080151423096696}

In [ ]:
timm.

In [ ]:
alpha = [0.75,0.65,0.65,0.4,0.65]
class Net(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='resnext50_32x4d',
                                    source='torchvision',
                                    pretrained=True,
                                    n_layers=-2,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder(1,5,self.encoder.output_dimension)
        #self.loss = nn.CrossEntropyLoss()
        #self.loss = FocalLoss(alpha=alpha, gamma=2)
        self.loss = FocalCosineLoss()
    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss


In [ ]:
class ENet(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = EfficientNet.from_pretrained('efficientnet-b4')
        self.encoder._fc = nn.Linear(self.encoder._fc.in_features,5)
        #self.loss = nn.CrossEntropyLoss()
        self.loss = FocalLoss(alpha=alpha, gamma=2)
    def forward(self, x, targets):
        logits = self.encoder(x)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [ ]:
class LitCassava(pl.LightningModule):
    def __init__(self, model):
        super(LitCassava, self).__init__()
        self.model = model
        self.metric = pl.metrics.Accuracy()
        self.learning_rate = 5e-6

    def forward(self, x, targets, *args, **kwargs):
        return self.model(x, targets)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.learning_rate, weight_decay=0.001)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=2)

        return (
            [optimizer],
            [{'scheduler': scheduler, 'interval': 'epoch', 'monitor': 'valid_loss'}],
        )

    def training_step(
        self, batch: torch.Tensor, batch_idx: int
    ) -> Union[int, Dict[str, Union[torch.Tensor, Dict[str, torch.Tensor]]]]:
        image = batch['image']
        target = batch['target']
        logits, loss = self(image, target)
        score = self.metric(logits.argmax(1), target)
        logs = {'train_loss': loss, f'train_accuracy': score}
        return {
            'loss': loss,
            'log': logs,
            'progress_bar': logs,
            'logits': logits,
            'target': target,
            f'train_accuracy': score,
        }

    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        y_true = torch.cat([x['target'] for x in outputs])
        y_pred = torch.cat([x['logits'] for x in outputs])
        score = self.metric(y_pred.argmax(1), y_true)
        
        logs = {'train_loss': avg_loss, 'train_accuracy': score}
        return {'log': logs, 'progress_bar': logs}

    def validation_step(
        self, batch: torch.Tensor, batch_idx: int
    ) -> Union[int, Dict[str, Union[torch.Tensor, Dict[str, torch.Tensor]]]]:
        image = batch['image']
        target = batch['target']
        logits, loss = self(image, target)
        score = self.metric(logits.argmax(1), target)
        logs = {'valid_loss': loss, f'valid_accuracy': score}

        return {
            'loss': loss,
            'log': logs,
            'progress_bar': logs,
            'logits': logits,
            'target': target,
            f'valid_accuracy': score,
        }

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        y_true = torch.cat([x['target'] for x in outputs])
        y_pred = torch.cat([x['logits'] for x in outputs])
        score = self.metric(y_pred.argmax(1), y_true)

        # score = torch.tensor(1.0, device=self.device)
        logs = {'valid_loss': avg_loss, f'valid_accuracy': score, 'accuracy': score}
        return {'valid_loss': avg_loss, 'log': logs, 'progress_bar': logs}

In [ ]:
#EfficientNet.from_pretrained('efficientnet-b4')

In [ ]:
#add reduce LR on plateau, cosine annealing LR, snapshot ensemble. 

In [ ]:
# savedModels = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\resnext50_32x4d_sz_448\0\epoch=13_valid_loss=0.3457_valid_accuracy=0.8888.ckpt',
#               1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\resnext50_32x4d_sz_448\1\epoch=14_valid_loss=0.3207_valid_accuracy=0.8956.ckpt',
#               2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\resnext50_32x4d_sz_448\2\epoch=11_valid_loss=0.3372_valid_accuracy=0.8904.ckpt',
#               3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\resnext50_32x4d_sz_448\3\epoch=13_valid_loss=0.3607_valid_accuracy=0.8848.ckpt',
#               4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\resnext50_32x4d_sz_448\4\epoch=14_valid_loss=0.3422_valid_accuracy=0.8857.ckpt'
#               }


savedModels = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\efficientnet-b4_sz_448\0\epoch=3_valid_loss=0.3757_valid_accuracy=0.8857.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\efficientnet-b4_sz_448\1\epoch=7_valid_loss=0.3375_valid_accuracy=0.8916.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\efficientnet-b4_sz_448\2\epoch=7_valid_loss=0.3340_valid_accuracy=0.8881.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\efficientnet-b4_sz_448\3\epoch=8_valid_loss=0.3813_valid_accuracy=0.8822.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\efficientnet-b4_sz_448\4\epoch=3_valid_loss=0.3785_valid_accuracy=0.8745.ckpt'
              }


In [ ]:
#a = torch.load(r'C:\Users\Kaggle\Leaf_Classification\saved_models\resnext50_32x4d_sz_448\4\epoch=14_valid_loss=0.3422_valid_accuracy=0.8857.ckpt')
# a['state_dict']['model.loss.nll_loss.weight'] = torch.tensor(0.2500)

In [ ]:
#a.keys()

In [ ]:
# model = Net()
# lit_model = LitCassava(model)
# lit_model.load_state_dict(a['state_dict'],strict=True)

In [ ]:
for f in range(5):
    print(f"Running Fold:{f}")
    model = ENet()
    dm = CassavaDataModule(labels, train_augs, valid_augs, f'{base_dir}/train_images/',bs=6,fold=f)
    
    modelSavePath = f'C:/Users/Kaggle/Leaf_Classification/saved_models/{f}'
    if not os.path.exists(modelSavePath):
        os.makedirs(modelSavePath)
        
    trainer = pl.Trainer(
        checkpoint_callback=ModelCheckpoint(monitor='valid_loss',
                                            save_top_k=2, dirpath = modelSavePath,filename='{epoch}_{valid_loss:.4f}_{valid_accuracy:.4f}', mode='min'),
        accumulate_grad_batches = 1,
        gpus=1,
        max_epochs=10,
        num_sanity_val_steps=0,
        weights_summary='top',
        precision = 32,
        callbacks = [EarlyStopping(monitor='valid_loss', patience=4, mode='min')]
    )
    
    lit_model = LitCassava(model)
    
#     checkpoint = torch.load(savedModels[f], map_location=lambda storage, loc: storage)
#     checkpoint['state_dict']['model.loss.nll_loss.weight'] = torch.tensor(alpha)
#     lit_model.load_state_dict(checkpoint['state_dict'],strict=True)
                              
    trainer.fit(lit_model, dm)

    del trainer,model,lit_model
    gc.collect()
    

In [ ]:
#model = Net()

In [ ]:
# fold = 1
# dm = CassavaDataModule(labels, train_augs, valid_augs, f'{base_dir}/train_images/',bs=96,fold=1)

In [ ]:
# modelSavePath = f'C:/Users/Kaggle/Leaf_Classification/saved_models/{fold}'

# if not os.path.exists(modelSavePath):
#     os.makedirs(modelSavePath)
    
# trainer = pl.Trainer(
#         checkpoint_callback=ModelCheckpoint(monitor='valid_loss',
#                                             save_top_k=2, dirpath = modelSavePath,filename='{epoch}_{valid_loss:.4f}_{valid_accuracy:.4f}', mode='min'),
#         gpus=1,
#         max_epochs=3,
#         num_sanity_val_steps=0,
#         weights_summary='top',
#         callbacks = [EarlyStopping(monitor='valid_loss', patience=4, mode='min')]
# )

In [ ]:
#lit_model = LitCassava(model)

In [ ]:
#trainer.fit(lit_model, dm)